In [ ]:
from pathlib import Path
from copy import deepcopy
import pandas as pd
import rpy2.rinterface
from math import floor
from warnings import filterwarnings
from IPython.display import Image
from random import shuffle
from PIL import Image as PImage
from PIL import ImageDraw, ImageFont
from mmeds.util import load_config

filterwarnings('ignore', category=rpy2.rinterface.RRuntimeWarning)

# Load the configuration
config = load_config(Path('config_file.yaml'), Path('metadata.tsv'), True)

# Load metadata file
if 'qiime2' == 'qiime2':
    mdf = pd.read_csv('qiime_mapping_file.tsv', skiprows=[1], sep='\t', dtype={'#SampleID': 'str'})
else:
    mdf = pd.read_csv('qiime_mapping_file.tsv', sep='\t')
mdf.set_index('#SampleID', inplace=True)

# Load the columns to use for analysis
metadata_columns = sorted(config['metadata'])
metadata_discrete = sorted([x for x in config['metadata'] if not config['metadata_continuous'][x]])
metadata_continuous = sorted([x for x in config['metadata'] if config['metadata_continuous'][x]])

# Calculate needed number of discrete colors, equal to the number of values across all discrete variables
max_colors = 0
for group_name in metadata_discrete:
    if group_name in mdf and not mdf[group_name].isnull().all():
        grouping = mdf[group_name]
        max_colors += grouping.nunique()

all_colors = ['color{}'.format(i) for i in range(max_colors)]

# Load the extention for jupyter
%load_ext rpy2.ipython

In [ ]:
%%R -i all_colors -o allRGB
library(ggplot2)
library(RColorBrewer)
library(GGally)
library(ggrepel)

# Create custom color palette from brewer "Set1"
myColors <- brewer.pal(8, "Set1")

# Extrapolate extra colors only if number of vars exceeds the number of colors in the initial palette
if (length(myColors) >= length(unique(all_colors))) {
    allColors <- myColors[0 : length(unique(all_colors))]
} else {
    colorMaker <- colorRampPalette(myColors)
    allColors <- colorMaker(length(unique(all_colors)))
}

# Rename the colors to match with the groups
names(allColors) <- all_colors

# Create the objects for graphing with the colors
colScale <- scale_color_manual(name = ~GroupID, values = allColors)
colFill <- scale_fill_manual(name = ~GroupID, values = allColors)

# Rename the colors to match with the groups
# Get the RGB values for the colors
allRGB <- data.frame(apply(data.frame(allColors), 1, col2rgb))

In [ ]:
# Assign colors to discrete values
color_maps = {}
color_max = 0
for group_name in metadata_discrete:
    if not mdf[group_name].isnull().all():
        grouping = mdf[group_name]
        color_maps[group_name] = {str(x):'color{}'.format(color_max + i)
                                      for i, x in enumerate(grouping.drop_duplicates())} 
        color_max += grouping.nunique() # Unique grouping vals

# Demultiplexing Summary

# Table Statistics Summary

# Taxonomy Summary

## Interpreting Taxonomy Results

Taxonomy plots represent the abundance of different taxa using stacked plots on a per-sample or per-group (averaged) basis. Data is normalized so that abundances per sample or per group add up to 100%. When using group-based taxonomy plots, it should be noted that only average abundances are shown per group and taxa: this can induce visual biases when a small number of samples in a group have significantly higher abundance of a given taxa compared to the rest of samples in the group, and give the (incorrect) impression that the group as a whole has high high abundance of the taxa.

## Phylum Level

In [ ]:
df = pd.read_csv('level-2.csv', sep=',')
# Remove unnecessary metadata
headers = list(filter(lambda x: 'k__' in x, df.columns)) + ['index']
df = df.filter(headers).set_index('index').T
# Calculate relative abundances
df = df.apply(lambda x: x / x.sum(), axis='index')
# Transpose the dataframe and convert to long format
df = df.T.reset_index(level=0).melt('index')
# Rename the columns
df.rename({'index': 'variable', 'variable': 'X.OTU.ID'}, axis='columns', inplace=True)
df = df.astype({'variable': 'str'})
# Modify the metadata
mdf_lite = mdf.reset_index()
mdf_lite = mdf_lite[['#SampleID', 'SpecimenBodySite']].rename({'#SampleID': 'variable'}, axis='columns')
# Merge with the data
df = df.merge(mdf_lite, how='inner')
df.SpecimenBodySite = df.SpecimenBodySite.astype(str)
# Add colors for plotting
otu_max_colors_Phylum = sorted(df['X.OTU.ID'].unique())

In [ ]:
%%R -i otu_max_colors_Phylum -o otuAllRGBPhylum

# Create custom color palette
otuColorsPhylum <- brewer.pal(12, "Paired")
otuColorMakerPhylum <- colorRampPalette(otuColorsPhylum)
otuAllColorsPhylum <- otuColorMakerPhylum(length(otu_max_colors_Phylum))
names(otuAllColorsPhylum) <- otu_max_colors_Phylum
# Create the functions for applying it to plots
otuColScalePhylum <- scale_color_manual(name = ~X.OTU.ID, values = otuAllColorsPhylum)
otuColFillPhylum <- scale_fill_manual(name = ~X.OTU.ID, values = otuAllColorsPhylum)
# Get the RGB values for the colors
otuAllRGBPhylum <- data.frame(apply(data.frame(otuAllColorsPhylum), 1, col2rgb))
names(otuAllRGBPhylum) <- otu_max_colors_Phylum

In [ ]:

with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i
# Filter by abundance
filtered_df = df[df.value > float(config['abundance_threshold'])]

groups = filtered_df.groupby('SpecimenBodySite')
latex = []
for name, group in groups:
    # Get the means for each group
    means = group.groupby('X.OTU.ID').mean()
    line_latex = []
    for row in means.itertuples():
        perc = 100 * float(row.value)
        # Align sample names
        if perc < 10:
            line_text = '{}/ {:.2f}'.format(row.Index.replace('_', '\_'), perc)
        else:
            line_text = '{}/{:.2f}'.format(row.Index.replace('_', '\_'), perc)
        line_latex.append(line_text)
    latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuPhylumSpecimenBodySite',
                                                          colors=','.join(line_latex)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

In [ ]:
%%R -i df
p <- ggplot(df, aes(x = variable, y = value, fill = X.OTU.ID)) +
     geom_bar(stat = "identity") +
     labs(x = "Sample ID",
          title = 'Taxa Summary',
          subtitle = 'Grouped by SpecimenBodySite') +
     scale_y_discrete(name = "OTU Percentage",
                      limits = c(0, 1),
                      expand = c(0, 0)) +
     theme(text = element_text(size = 8.5,
                               face = "bold"),
           element_line(size = 0.1),
           legend.position = "none",
           axis.text.x = element_text(angle = 300),
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5)) +
     guides(fill = guide_legend(ncol = 4)) +
     facet_grid(~SpecimenBodySite, scales = "free", space = "free") +
     otuColScalePhylum + otuColFillPhylum

ggsave("level-2-SpecimenBodySite.png", height = 6, width = 8)

In [ ]:
Image("level-2-SpecimenBodySite.png")

In [ ]:
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i


# Get the mean percentage of each taxon
otu_percs = df[df.value > float(config['abundance_threshold'])].groupby('X.OTU.ID').mean()

entries = []
# Remove duplicate values
for row in otu_percs.itertuples():
    otu = row.Index

    # Get the color
    RGB = tuple(otuAllRGBPhylum[otu])

    # Align sample names
    perc = 100 * float(otu_percs.loc[otu]['value'])
    if perc < 1:
        perc_text = '<0.01'
    elif perc < 10:
        perc_text = ' {:.2f}'.format(perc)
    else:
        perc_text = '{:.2f}'.format(perc)

    entries.append([otu, RGB, perc_text])


# Create the latex for the legends
latex = []
colors = []
for key, value, perc in entries:
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key.replace('_', '\_'),
                                                                      r=value[0],
                                                                      g=value[1],
                                                                      b=value[2]
                                                                     ))
    colors.append('{}/{}'.format(key.replace('_', '\_'), perc))


latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuPhylum',
                                                      colors=','.join(colors)))
new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)


The above plot represents the percentage of each sample belonging to particular taxon summarized at the Phylum level.

\pagebreak

In [ ]:
df = pd.read_csv('level-2.csv', sep=',')
# Remove unnecessary metadata
headers = list(filter(lambda x: 'k__' in x, df.columns)) + ['index']
df = df.filter(headers).set_index('index').T
# Calculate relative abundances
df = df.apply(lambda x: x / x.sum(), axis='index')
# Transpose the dataframe and convert to long format
df = df.T.reset_index(level=0).melt('index')
# Rename the columns
df.rename({'index': 'variable', 'variable': 'X.OTU.ID'}, axis='columns', inplace=True)
df = df.astype({'variable': 'str'})
# Modify the metadata
mdf_lite = mdf.reset_index()
mdf_lite = mdf_lite[['#SampleID', 'SpecimenType']].rename({'#SampleID': 'variable'}, axis='columns')
# Merge with the data
df = df.merge(mdf_lite, how='inner')
df.SpecimenType = df.SpecimenType.astype(str)
# Add colors for plotting
otu_max_colors_Phylum = sorted(df['X.OTU.ID'].unique())

In [ ]:

with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i
# Filter by abundance
filtered_df = df[df.value > float(config['abundance_threshold'])]

groups = filtered_df.groupby('SpecimenType')
latex = []
for name, group in groups:
    # Get the means for each group
    means = group.groupby('X.OTU.ID').mean()
    line_latex = []
    for row in means.itertuples():
        perc = 100 * float(row.value)
        # Align sample names
        if perc < 10:
            line_text = '{}/ {:.2f}'.format(row.Index.replace('_', '\_'), perc)
        else:
            line_text = '{}/{:.2f}'.format(row.Index.replace('_', '\_'), perc)
        line_latex.append(line_text)
    latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuPhylumSpecimenType',
                                                          colors=','.join(line_latex)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

In [ ]:
%%R -i df
p <- ggplot(df, aes(x = variable, y = value, fill = X.OTU.ID)) +
     geom_bar(stat = "identity") +
     labs(x = "Sample ID",
          title = 'Taxa Summary',
          subtitle = 'Grouped by SpecimenType') +
     scale_y_discrete(name = "OTU Percentage",
                      limits = c(0, 1),
                      expand = c(0, 0)) +
     theme(text = element_text(size = 8.5,
                               face = "bold"),
           element_line(size = 0.1),
           legend.position = "none",
           axis.text.x = element_text(angle = 300),
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5)) +
     guides(fill = guide_legend(ncol = 4)) +
     facet_grid(~SpecimenType, scales = "free", space = "free") +
     otuColScalePhylum + otuColFillPhylum

ggsave("level-2-SpecimenType.png", height = 6, width = 8)

In [ ]:
Image("level-2-SpecimenType.png")

In [ ]:
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i


# Get the mean percentage of each taxon
otu_percs = df[df.value > float(config['abundance_threshold'])].groupby('X.OTU.ID').mean()

entries = []
# Remove duplicate values
for row in otu_percs.itertuples():
    otu = row.Index

    # Get the color
    RGB = tuple(otuAllRGBPhylum[otu])

    # Align sample names
    perc = 100 * float(otu_percs.loc[otu]['value'])
    if perc < 1:
        perc_text = '<0.01'
    elif perc < 10:
        perc_text = ' {:.2f}'.format(perc)
    else:
        perc_text = '{:.2f}'.format(perc)

    entries.append([otu, RGB, perc_text])


# Create the latex for the legends
latex = []
colors = []
for key, value, perc in entries:
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key.replace('_', '\_'),
                                                                      r=value[0],
                                                                      g=value[1],
                                                                      b=value[2]
                                                                     ))
    colors.append('{}/{}'.format(key.replace('_', '\_'), perc))


latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuPhylum',
                                                      colors=','.join(colors)))
new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)


The above plot represents the percentage of each sample belonging to particular taxon summarized at the Phylum level.

\pagebreak

In [ ]:
df = pd.read_csv('level-2.csv', sep=',')
# Remove unnecessary metadata
headers = list(filter(lambda x: 'k__' in x, df.columns)) + ['index']
df = df.filter(headers).set_index('index').T
# Calculate relative abundances
df = df.apply(lambda x: x / x.sum(), axis='index')
# Transpose the dataframe and convert to long format
df = df.T.reset_index(level=0).melt('index')
# Rename the columns
df.rename({'index': 'variable', 'variable': 'X.OTU.ID'}, axis='columns', inplace=True)
df = df.astype({'variable': 'str'})
# Modify the metadata
mdf_lite = mdf.reset_index()
mdf_lite = mdf_lite[['#SampleID', 'Ethnicity']].rename({'#SampleID': 'variable'}, axis='columns')
# Merge with the data
df = df.merge(mdf_lite, how='inner')
df.Ethnicity = df.Ethnicity.astype(str)
# Add colors for plotting
otu_max_colors_Phylum = sorted(df['X.OTU.ID'].unique())

In [ ]:

with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i
# Filter by abundance
filtered_df = df[df.value > float(config['abundance_threshold'])]

groups = filtered_df.groupby('Ethnicity')
latex = []
for name, group in groups:
    # Get the means for each group
    means = group.groupby('X.OTU.ID').mean()
    line_latex = []
    for row in means.itertuples():
        perc = 100 * float(row.value)
        # Align sample names
        if perc < 10:
            line_text = '{}/ {:.2f}'.format(row.Index.replace('_', '\_'), perc)
        else:
            line_text = '{}/{:.2f}'.format(row.Index.replace('_', '\_'), perc)
        line_latex.append(line_text)
    latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuPhylumEthnicity',
                                                          colors=','.join(line_latex)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

In [ ]:
%%R -i df
p <- ggplot(df, aes(x = variable, y = value, fill = X.OTU.ID)) +
     geom_bar(stat = "identity") +
     labs(x = "Sample ID",
          title = 'Taxa Summary',
          subtitle = 'Grouped by Ethnicity') +
     scale_y_discrete(name = "OTU Percentage",
                      limits = c(0, 1),
                      expand = c(0, 0)) +
     theme(text = element_text(size = 8.5,
                               face = "bold"),
           element_line(size = 0.1),
           legend.position = "none",
           axis.text.x = element_text(angle = 300),
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5)) +
     guides(fill = guide_legend(ncol = 4)) +
     facet_grid(~Ethnicity, scales = "free", space = "free") +
     otuColScalePhylum + otuColFillPhylum

ggsave("level-2-Ethnicity.png", height = 6, width = 8)

In [ ]:
Image("level-2-Ethnicity.png")

In [ ]:
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i


# Get the mean percentage of each taxon
otu_percs = df[df.value > float(config['abundance_threshold'])].groupby('X.OTU.ID').mean()

entries = []
# Remove duplicate values
for row in otu_percs.itertuples():
    otu = row.Index

    # Get the color
    RGB = tuple(otuAllRGBPhylum[otu])

    # Align sample names
    perc = 100 * float(otu_percs.loc[otu]['value'])
    if perc < 1:
        perc_text = '<0.01'
    elif perc < 10:
        perc_text = ' {:.2f}'.format(perc)
    else:
        perc_text = '{:.2f}'.format(perc)

    entries.append([otu, RGB, perc_text])


# Create the latex for the legends
latex = []
colors = []
for key, value, perc in entries:
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key.replace('_', '\_'),
                                                                      r=value[0],
                                                                      g=value[1],
                                                                      b=value[2]
                                                                     ))
    colors.append('{}/{}'.format(key.replace('_', '\_'), perc))


latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuPhylum',
                                                      colors=','.join(colors)))
new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)


The above plot represents the percentage of each sample belonging to particular taxon summarized at the Phylum level.

\pagebreak

In [ ]:
df = pd.read_csv('level-2.csv', sep=',')
# Remove unnecessary metadata
headers = list(filter(lambda x: 'k__' in x, df.columns)) + ['index']
df = df.filter(headers).set_index('index').T
# Calculate relative abundances
df = df.apply(lambda x: x / x.sum(), axis='index')
# Transpose the dataframe and convert to long format
df = df.T.reset_index(level=0).melt('index')
# Rename the columns
df.rename({'index': 'variable', 'variable': 'X.OTU.ID'}, axis='columns', inplace=True)
df = df.astype({'variable': 'str'})
# Modify the metadata
mdf_lite = mdf.reset_index()
mdf_lite = mdf_lite[['#SampleID', 'InterventionCode']].rename({'#SampleID': 'variable'}, axis='columns')
# Merge with the data
df = df.merge(mdf_lite, how='inner')
df.InterventionCode = df.InterventionCode.astype(str)
# Add colors for plotting
otu_max_colors_Phylum = sorted(df['X.OTU.ID'].unique())

In [ ]:

with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i
# Filter by abundance
filtered_df = df[df.value > float(config['abundance_threshold'])]

groups = filtered_df.groupby('InterventionCode')
latex = []
for name, group in groups:
    # Get the means for each group
    means = group.groupby('X.OTU.ID').mean()
    line_latex = []
    for row in means.itertuples():
        perc = 100 * float(row.value)
        # Align sample names
        if perc < 10:
            line_text = '{}/ {:.2f}'.format(row.Index.replace('_', '\_'), perc)
        else:
            line_text = '{}/{:.2f}'.format(row.Index.replace('_', '\_'), perc)
        line_latex.append(line_text)
    latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuPhylumInterventionCode',
                                                          colors=','.join(line_latex)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

In [ ]:
%%R -i df
p <- ggplot(df, aes(x = variable, y = value, fill = X.OTU.ID)) +
     geom_bar(stat = "identity") +
     labs(x = "Sample ID",
          title = 'Taxa Summary',
          subtitle = 'Grouped by InterventionCode') +
     scale_y_discrete(name = "OTU Percentage",
                      limits = c(0, 1),
                      expand = c(0, 0)) +
     theme(text = element_text(size = 8.5,
                               face = "bold"),
           element_line(size = 0.1),
           legend.position = "none",
           axis.text.x = element_text(angle = 300),
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5)) +
     guides(fill = guide_legend(ncol = 4)) +
     facet_grid(~InterventionCode, scales = "free", space = "free") +
     otuColScalePhylum + otuColFillPhylum

ggsave("level-2-InterventionCode.png", height = 6, width = 8)

In [ ]:
Image("level-2-InterventionCode.png")

In [ ]:
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i


# Get the mean percentage of each taxon
otu_percs = df[df.value > float(config['abundance_threshold'])].groupby('X.OTU.ID').mean()

entries = []
# Remove duplicate values
for row in otu_percs.itertuples():
    otu = row.Index

    # Get the color
    RGB = tuple(otuAllRGBPhylum[otu])

    # Align sample names
    perc = 100 * float(otu_percs.loc[otu]['value'])
    if perc < 1:
        perc_text = '<0.01'
    elif perc < 10:
        perc_text = ' {:.2f}'.format(perc)
    else:
        perc_text = '{:.2f}'.format(perc)

    entries.append([otu, RGB, perc_text])


# Create the latex for the legends
latex = []
colors = []
for key, value, perc in entries:
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key.replace('_', '\_'),
                                                                      r=value[0],
                                                                      g=value[1],
                                                                      b=value[2]
                                                                     ))
    colors.append('{}/{}'.format(key.replace('_', '\_'), perc))


latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuPhylum',
                                                      colors=','.join(colors)))
new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)


The above plot represents the percentage of each sample belonging to particular taxon summarized at the Phylum level.

\pagebreak

## Genus Level

In [ ]:
df = pd.read_csv('level-6.csv', sep=',')
# Remove unnecessary metadata
headers = list(filter(lambda x: 'k__' in x, df.columns)) + ['index']
df = df.filter(headers).set_index('index').T
# Calculate relative abundances
df = df.apply(lambda x: x / x.sum(), axis='index')
# Transpose the dataframe and convert to long format
df = df.T.reset_index(level=0).melt('index')
# Rename the columns
df.rename({'index': 'variable', 'variable': 'X.OTU.ID'}, axis='columns', inplace=True)
df = df.astype({'variable': 'str'})
# Modify the metadata
mdf_lite = mdf.reset_index()
mdf_lite = mdf_lite[['#SampleID', 'SpecimenBodySite']].rename({'#SampleID': 'variable'}, axis='columns')
# Merge with the data
df = df.merge(mdf_lite, how='inner')
df.SpecimenBodySite = df.SpecimenBodySite.astype(str)
# Add colors for plotting
otu_max_colors_Genus = sorted(df['X.OTU.ID'].unique())

In [ ]:
%%R -i otu_max_colors_Genus -o otuAllRGBGenus

# Create custom color palette
otuColorsGenus <- brewer.pal(12, "Paired")
otuColorMakerGenus <- colorRampPalette(otuColorsGenus)
otuAllColorsGenus <- otuColorMakerGenus(length(otu_max_colors_Genus))
names(otuAllColorsGenus) <- otu_max_colors_Genus
# Create the functions for applying it to plots
otuColScaleGenus <- scale_color_manual(name = ~X.OTU.ID, values = otuAllColorsGenus)
otuColFillGenus <- scale_fill_manual(name = ~X.OTU.ID, values = otuAllColorsGenus)
# Get the RGB values for the colors
otuAllRGBGenus <- data.frame(apply(data.frame(otuAllColorsGenus), 1, col2rgb))
names(otuAllRGBGenus) <- otu_max_colors_Genus

In [ ]:

with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i
# Filter by abundance
filtered_df = df[df.value > float(config['abundance_threshold'])]

groups = filtered_df.groupby('SpecimenBodySite')
latex = []
for name, group in groups:
    # Get the means for each group
    means = group.groupby('X.OTU.ID').mean()
    line_latex = []
    for row in means.itertuples():
        perc = 100 * float(row.value)
        # Align sample names
        if perc < 10:
            line_text = '{}/ {:.2f}'.format(row.Index.replace('_', '\_'), perc)
        else:
            line_text = '{}/{:.2f}'.format(row.Index.replace('_', '\_'), perc)
        line_latex.append(line_text)
    latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuGenusSpecimenBodySite',
                                                          colors=','.join(line_latex)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

In [ ]:
%%R -i df
p <- ggplot(df, aes(x = variable, y = value, fill = X.OTU.ID)) +
     geom_bar(stat = "identity") +
     labs(x = "Sample ID",
          title = 'Taxa Summary',
          subtitle = 'Grouped by SpecimenBodySite') +
     scale_y_discrete(name = "OTU Percentage",
                      limits = c(0, 1),
                      expand = c(0, 0)) +
     theme(text = element_text(size = 8.5,
                               face = "bold"),
           element_line(size = 0.1),
           legend.position = "none",
           axis.text.x = element_text(angle = 300),
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5)) +
     guides(fill = guide_legend(ncol = 4)) +
     facet_grid(~SpecimenBodySite, scales = "free", space = "free") +
     otuColScaleGenus + otuColFillGenus

ggsave("level-6-SpecimenBodySite.png", height = 6, width = 8)

In [ ]:
Image("level-6-SpecimenBodySite.png")

In [ ]:
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i


# Get the mean percentage of each taxon
otu_percs = df[df.value > float(config['abundance_threshold'])].groupby('X.OTU.ID').mean()

entries = []
# Remove duplicate values
for row in otu_percs.itertuples():
    otu = row.Index

    # Get the color
    RGB = tuple(otuAllRGBGenus[otu])

    # Align sample names
    perc = 100 * float(otu_percs.loc[otu]['value'])
    if perc < 1:
        perc_text = '<0.01'
    elif perc < 10:
        perc_text = ' {:.2f}'.format(perc)
    else:
        perc_text = '{:.2f}'.format(perc)

    entries.append([otu, RGB, perc_text])


# Create the latex for the legends
latex = []
colors = []
for key, value, perc in entries:
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key.replace('_', '\_'),
                                                                      r=value[0],
                                                                      g=value[1],
                                                                      b=value[2]
                                                                     ))
    colors.append('{}/{}'.format(key.replace('_', '\_'), perc))


latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuGenus',
                                                      colors=','.join(colors)))
new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)


The above plot represents the percentage of each sample belonging to particular taxon summarized at the Genus level.

\pagebreak

In [ ]:
df = pd.read_csv('level-6.csv', sep=',')
# Remove unnecessary metadata
headers = list(filter(lambda x: 'k__' in x, df.columns)) + ['index']
df = df.filter(headers).set_index('index').T
# Calculate relative abundances
df = df.apply(lambda x: x / x.sum(), axis='index')
# Transpose the dataframe and convert to long format
df = df.T.reset_index(level=0).melt('index')
# Rename the columns
df.rename({'index': 'variable', 'variable': 'X.OTU.ID'}, axis='columns', inplace=True)
df = df.astype({'variable': 'str'})
# Modify the metadata
mdf_lite = mdf.reset_index()
mdf_lite = mdf_lite[['#SampleID', 'SpecimenType']].rename({'#SampleID': 'variable'}, axis='columns')
# Merge with the data
df = df.merge(mdf_lite, how='inner')
df.SpecimenType = df.SpecimenType.astype(str)
# Add colors for plotting
otu_max_colors_Genus = sorted(df['X.OTU.ID'].unique())

In [ ]:

with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i
# Filter by abundance
filtered_df = df[df.value > float(config['abundance_threshold'])]

groups = filtered_df.groupby('SpecimenType')
latex = []
for name, group in groups:
    # Get the means for each group
    means = group.groupby('X.OTU.ID').mean()
    line_latex = []
    for row in means.itertuples():
        perc = 100 * float(row.value)
        # Align sample names
        if perc < 10:
            line_text = '{}/ {:.2f}'.format(row.Index.replace('_', '\_'), perc)
        else:
            line_text = '{}/{:.2f}'.format(row.Index.replace('_', '\_'), perc)
        line_latex.append(line_text)
    latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuGenusSpecimenType',
                                                          colors=','.join(line_latex)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

In [ ]:
%%R -i df
p <- ggplot(df, aes(x = variable, y = value, fill = X.OTU.ID)) +
     geom_bar(stat = "identity") +
     labs(x = "Sample ID",
          title = 'Taxa Summary',
          subtitle = 'Grouped by SpecimenType') +
     scale_y_discrete(name = "OTU Percentage",
                      limits = c(0, 1),
                      expand = c(0, 0)) +
     theme(text = element_text(size = 8.5,
                               face = "bold"),
           element_line(size = 0.1),
           legend.position = "none",
           axis.text.x = element_text(angle = 300),
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5)) +
     guides(fill = guide_legend(ncol = 4)) +
     facet_grid(~SpecimenType, scales = "free", space = "free") +
     otuColScaleGenus + otuColFillGenus

ggsave("level-6-SpecimenType.png", height = 6, width = 8)

In [ ]:
Image("level-6-SpecimenType.png")

In [ ]:
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i


# Get the mean percentage of each taxon
otu_percs = df[df.value > float(config['abundance_threshold'])].groupby('X.OTU.ID').mean()

entries = []
# Remove duplicate values
for row in otu_percs.itertuples():
    otu = row.Index

    # Get the color
    RGB = tuple(otuAllRGBGenus[otu])

    # Align sample names
    perc = 100 * float(otu_percs.loc[otu]['value'])
    if perc < 1:
        perc_text = '<0.01'
    elif perc < 10:
        perc_text = ' {:.2f}'.format(perc)
    else:
        perc_text = '{:.2f}'.format(perc)

    entries.append([otu, RGB, perc_text])


# Create the latex for the legends
latex = []
colors = []
for key, value, perc in entries:
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key.replace('_', '\_'),
                                                                      r=value[0],
                                                                      g=value[1],
                                                                      b=value[2]
                                                                     ))
    colors.append('{}/{}'.format(key.replace('_', '\_'), perc))


latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuGenus',
                                                      colors=','.join(colors)))
new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)


The above plot represents the percentage of each sample belonging to particular taxon summarized at the Genus level.

\pagebreak

In [ ]:
df = pd.read_csv('level-6.csv', sep=',')
# Remove unnecessary metadata
headers = list(filter(lambda x: 'k__' in x, df.columns)) + ['index']
df = df.filter(headers).set_index('index').T
# Calculate relative abundances
df = df.apply(lambda x: x / x.sum(), axis='index')
# Transpose the dataframe and convert to long format
df = df.T.reset_index(level=0).melt('index')
# Rename the columns
df.rename({'index': 'variable', 'variable': 'X.OTU.ID'}, axis='columns', inplace=True)
df = df.astype({'variable': 'str'})
# Modify the metadata
mdf_lite = mdf.reset_index()
mdf_lite = mdf_lite[['#SampleID', 'Ethnicity']].rename({'#SampleID': 'variable'}, axis='columns')
# Merge with the data
df = df.merge(mdf_lite, how='inner')
df.Ethnicity = df.Ethnicity.astype(str)
# Add colors for plotting
otu_max_colors_Genus = sorted(df['X.OTU.ID'].unique())

In [ ]:

with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i
# Filter by abundance
filtered_df = df[df.value > float(config['abundance_threshold'])]

groups = filtered_df.groupby('Ethnicity')
latex = []
for name, group in groups:
    # Get the means for each group
    means = group.groupby('X.OTU.ID').mean()
    line_latex = []
    for row in means.itertuples():
        perc = 100 * float(row.value)
        # Align sample names
        if perc < 10:
            line_text = '{}/ {:.2f}'.format(row.Index.replace('_', '\_'), perc)
        else:
            line_text = '{}/{:.2f}'.format(row.Index.replace('_', '\_'), perc)
        line_latex.append(line_text)
    latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuGenusEthnicity',
                                                          colors=','.join(line_latex)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

In [ ]:
%%R -i df
p <- ggplot(df, aes(x = variable, y = value, fill = X.OTU.ID)) +
     geom_bar(stat = "identity") +
     labs(x = "Sample ID",
          title = 'Taxa Summary',
          subtitle = 'Grouped by Ethnicity') +
     scale_y_discrete(name = "OTU Percentage",
                      limits = c(0, 1),
                      expand = c(0, 0)) +
     theme(text = element_text(size = 8.5,
                               face = "bold"),
           element_line(size = 0.1),
           legend.position = "none",
           axis.text.x = element_text(angle = 300),
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5)) +
     guides(fill = guide_legend(ncol = 4)) +
     facet_grid(~Ethnicity, scales = "free", space = "free") +
     otuColScaleGenus + otuColFillGenus

ggsave("level-6-Ethnicity.png", height = 6, width = 8)

In [ ]:
Image("level-6-Ethnicity.png")

In [ ]:
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i


# Get the mean percentage of each taxon
otu_percs = df[df.value > float(config['abundance_threshold'])].groupby('X.OTU.ID').mean()

entries = []
# Remove duplicate values
for row in otu_percs.itertuples():
    otu = row.Index

    # Get the color
    RGB = tuple(otuAllRGBGenus[otu])

    # Align sample names
    perc = 100 * float(otu_percs.loc[otu]['value'])
    if perc < 1:
        perc_text = '<0.01'
    elif perc < 10:
        perc_text = ' {:.2f}'.format(perc)
    else:
        perc_text = '{:.2f}'.format(perc)

    entries.append([otu, RGB, perc_text])


# Create the latex for the legends
latex = []
colors = []
for key, value, perc in entries:
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key.replace('_', '\_'),
                                                                      r=value[0],
                                                                      g=value[1],
                                                                      b=value[2]
                                                                     ))
    colors.append('{}/{}'.format(key.replace('_', '\_'), perc))


latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuGenus',
                                                      colors=','.join(colors)))
new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)


The above plot represents the percentage of each sample belonging to particular taxon summarized at the Genus level.

\pagebreak

In [ ]:
df = pd.read_csv('level-6.csv', sep=',')
# Remove unnecessary metadata
headers = list(filter(lambda x: 'k__' in x, df.columns)) + ['index']
df = df.filter(headers).set_index('index').T
# Calculate relative abundances
df = df.apply(lambda x: x / x.sum(), axis='index')
# Transpose the dataframe and convert to long format
df = df.T.reset_index(level=0).melt('index')
# Rename the columns
df.rename({'index': 'variable', 'variable': 'X.OTU.ID'}, axis='columns', inplace=True)
df = df.astype({'variable': 'str'})
# Modify the metadata
mdf_lite = mdf.reset_index()
mdf_lite = mdf_lite[['#SampleID', 'InterventionCode']].rename({'#SampleID': 'variable'}, axis='columns')
# Merge with the data
df = df.merge(mdf_lite, how='inner')
df.InterventionCode = df.InterventionCode.astype(str)
# Add colors for plotting
otu_max_colors_Genus = sorted(df['X.OTU.ID'].unique())

In [ ]:

with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i
# Filter by abundance
filtered_df = df[df.value > float(config['abundance_threshold'])]

groups = filtered_df.groupby('InterventionCode')
latex = []
for name, group in groups:
    # Get the means for each group
    means = group.groupby('X.OTU.ID').mean()
    line_latex = []
    for row in means.itertuples():
        perc = 100 * float(row.value)
        # Align sample names
        if perc < 10:
            line_text = '{}/ {:.2f}'.format(row.Index.replace('_', '\_'), perc)
        else:
            line_text = '{}/{:.2f}'.format(row.Index.replace('_', '\_'), perc)
        line_latex.append(line_text)
    latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuGenusInterventionCode',
                                                          colors=','.join(line_latex)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

In [ ]:
%%R -i df
p <- ggplot(df, aes(x = variable, y = value, fill = X.OTU.ID)) +
     geom_bar(stat = "identity") +
     labs(x = "Sample ID",
          title = 'Taxa Summary',
          subtitle = 'Grouped by InterventionCode') +
     scale_y_discrete(name = "OTU Percentage",
                      limits = c(0, 1),
                      expand = c(0, 0)) +
     theme(text = element_text(size = 8.5,
                               face = "bold"),
           element_line(size = 0.1),
           legend.position = "none",
           axis.text.x = element_text(angle = 300),
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5)) +
     guides(fill = guide_legend(ncol = 4)) +
     facet_grid(~InterventionCode, scales = "free", space = "free") +
     otuColScaleGenus + otuColFillGenus

ggsave("level-6-InterventionCode.png", height = 6, width = 8)

In [ ]:
Image("level-6-InterventionCode.png")

In [ ]:
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* block packages *))' in line:
        packages_start = i
    elif '((* endblock packages *))' in line:
        packages_end = i


# Get the mean percentage of each taxon
otu_percs = df[df.value > float(config['abundance_threshold'])].groupby('X.OTU.ID').mean()

entries = []
# Remove duplicate values
for row in otu_percs.itertuples():
    otu = row.Index

    # Get the color
    RGB = tuple(otuAllRGBGenus[otu])

    # Align sample names
    perc = 100 * float(otu_percs.loc[otu]['value'])
    if perc < 1:
        perc_text = '<0.01'
    elif perc < 10:
        perc_text = ' {:.2f}'.format(perc)
    else:
        perc_text = '{:.2f}'.format(perc)

    entries.append([otu, RGB, perc_text])


# Create the latex for the legends
latex = []
colors = []
for key, value, perc in entries:
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key.replace('_', '\_'),
                                                                      r=value[0],
                                                                      g=value[1],
                                                                      b=value[2]
                                                                     ))
    colors.append('{}/{}'.format(key.replace('_', '\_'), perc))


latex.append('\\def\\{colorset}{{{colors}}}\n'.format(colorset='otuGenus',
                                                      colors=','.join(colors)))
new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)


The above plot represents the percentage of each sample belonging to particular taxon summarized at the Genus level.

\pagebreak

In [ ]:
# Reset this variable
latex = []
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* endblock packages *))' in line:
        packages_end = i

for key, value in allRGB.items():
    latex.append('\\definecolor{{{key}}}{{RGB}}{{{r},{g},{b}}}\n'.format(key=key,
                                                                         r=value[0],
                                                                         g=value[1],
                                                                         b=value[2]))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

# Alpha Diversity

## Interpreting Alpha Diversity Results

Alpha diversity estimates the amount of microbial diversity present in a sample or group of samples. There are several measures that can be used for alpha diversity, including observed features, Shannon's diversity or Faith's phylogenetic diversity. Because diversity estimates depend on the total number of sequences assigned to each sample, rarefaction curves are constructed to show the relation between alpha diversity (on the vertical axis) and sequencing depth (on the horizontal axis). Curves that gradually plateau as sequencing depth increases suggest that additional sequencing effort would not substantially yield additional results in terms of currently not observed diversity; curves that continue to increase suggest additional sequencing effort might be required to saturate the estimate.

## Shannon Diversity

In [ ]:
# Read in the data
df = pd.read_csv('shannon.csv', sep=',')

# Reshape the data into (mostly) long format
headers = list(set(mdf.columns).intersection(set(df.columns)))
df = df.melt(id_vars=['sample-id'] + headers)

# Remove info on specific iterations to allow for grouping by value
replacements = {x:int(x.split('_')[0].split('-')[1]) for x in df.variable.unique()}
df.variable.replace(replacements, inplace=True)

# For storing
group_means = []

for group_name in metadata_continuous:
    # Remove the metadata not relevant to this grouping
    groups = df[['sample-id', 'variable', 'value', group_name]]

    # Calculate the means accross iterations
    agger = {'value': 'mean', group_name: 'first'}
    groups = groups.groupby(['sample-id', 'variable']).agg(agger).reset_index()

    # Add a column to store the errors
    groups = groups.assign(Error=groups.value)

    # Group by metadata value and calculate the means and error
    agger = {'Error': 'sem', 'value': 'mean'}
    group = groups.groupby([group_name, 'variable']).agg(agger).reset_index()

    # Assign information for grouping
    group_names = [group_name for x in group[group_name]]
    group = group.assign(GroupName=group_names)

    # Rename columns and append to the list of dataframes
    new_names = {
        'variable': 'SamplingDepth',
        'value': 'AverageValue',
         group_name: 'Grouping'
    }
    group_means.append(group.rename(index=str, columns=new_names))

# Stack all the different groups into a single dataframe
df = pd.concat(group_means, axis=0, sort=False)
df.SamplingDepth = df.SamplingDepth.astype(float)
df.Error = df.Error.astype(float)
df.AverageValue = df.AverageValue.astype(float)
df.GroupName = df.GroupName.astype(str)

In [ ]:
%%R -i df

pd <- position_dodge(width = 50)
df.var <- subset(df, GroupName=="DaysSinceExperimentStart")
p <- ggplot(data = df.var, aes(x = SamplingDepth, y = AverageValue, color = Grouping,group=Grouping)) +
      geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
      geom_point(stat='identity', position = pd, size = 1) +
      geom_line(stat='identity', position = pd) +
      labs(title = 'Alpha Diversity',
           subtitle = 'Grouped by DaysSinceExperimentStart') +
      theme_bw() +
      theme(legend.position = 'bottom',
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5)) + scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

# Save plots
ggsave('shannon_DaysSinceExperimentStart.png', height = 6, width = 6)

In [ ]:
Image("shannon_DaysSinceExperimentStart.png")

\pagebreak

In [ ]:
%%R -i df

pd <- position_dodge(width = 50)
df.var <- subset(df, GroupName=="Depth")
p <- ggplot(data = df.var, aes(x = SamplingDepth, y = AverageValue, color = Grouping,group=Grouping)) +
      geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
      geom_point(stat='identity', position = pd, size = 1) +
      geom_line(stat='identity', position = pd) +
      labs(title = 'Alpha Diversity',
           subtitle = 'Grouped by Depth') +
      theme_bw() +
      theme(legend.position = 'bottom',
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5)) + scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

# Save plots
ggsave('shannon_Depth.png', height = 6, width = 6)

In [ ]:
Image("shannon_Depth.png")

\pagebreak

In [ ]:
# Read in the data
df = pd.read_csv('shannon.csv', sep=',')

# Reshape the data into (mostly) long format
headers = list(set(mdf.columns).intersection(set(df.columns)))
df = df.melt(id_vars=['sample-id'] + headers)

# Remove info on specific iterations to allow for grouping by value
replacements = {x:int(x.split('_')[0].split('-')[1]) for x in df.variable.unique()}
df.variable.replace(replacements, inplace=True)

# For storing
group_means = []

for group_name in metadata_discrete:
    # Remove the metadata not relevant to this grouping
    groups = df[['sample-id', 'variable', 'value', group_name]]

    # Calculate the means accross iterations
    agger = {'value': 'mean', group_name: 'first'}
    groups = groups.groupby(['sample-id', 'variable']).agg(agger).reset_index()

    # Add a column to store the errors
    groups = groups.assign(Error=groups.value)

    # Group by metadata value and calculate the means and error
    agger = {'Error': 'sem', 'value': 'mean'}
    group = groups.groupby([group_name, 'variable']).agg(agger).reset_index()

    # Assign information for the colors
    colors = [color_maps[group_name][str(x)] for x in group[group_name]]
    group = group.assign(GroupID=colors)

    # Assign information for grouping
    group_names = [group_name for x in group[group_name]]
    group = group.assign(GroupName=group_names)

    # Rename columns and append to the list of dataframes
    new_names = {
        'variable': 'SamplingDepth',
        'value': 'AverageValue',
         group_name: 'Grouping'
    }
    group_means.append(group.rename(index=str, columns=new_names))

# Stack all the different groups into a single dataframe
df = pd.concat(group_means, axis=0, sort=False)
df.SamplingDepth = df.SamplingDepth.astype(float)
df.Error = df.Error.astype(float)
df.AverageValue = df.AverageValue.astype(float)
df.Grouping = df.Grouping.astype(str)
df.GroupID = df.GroupID.astype(str)
df.GroupName = df.GroupName.astype(str)

In [ ]:
%%R -i df
pd <- position_dodge(width = 50)

p <- ggplot(data = df, aes(x = SamplingDepth, y = AverageValue, color = GroupID)) +
     geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
     geom_point(stat='identity', position = pd, size = 1) +
     geom_line(stat='identity', position = pd) +
     facet_wrap(~GroupName) + colFill + colScale +
     labs(title = 'Alpha Diversity',
          subtitle = 'Grouped by Discrete Metadata Categories') +
     theme_bw() +
     theme(legend.position = 'none',
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5))

# Save plots
ggsave('shannon_Depth.png', height = 6, width = 6)

In [ ]:
Image("shannon_Depth.png")

The above plot represents the average value of alpha diversity at each sampling depth. The error bars show the standard error within each group. Groups are determined by the metadata value in each category specified in the plot.

\pagebreak

## Faith's Phylogenetic Diversity

In [ ]:
# Read in the data
df = pd.read_csv('faith_pd.csv', sep=',')

# Reshape the data into (mostly) long format
headers = list(set(mdf.columns).intersection(set(df.columns)))
df = df.melt(id_vars=['sample-id'] + headers)

# Remove info on specific iterations to allow for grouping by value
replacements = {x:int(x.split('_')[0].split('-')[1]) for x in df.variable.unique()}
df.variable.replace(replacements, inplace=True)

# For storing
group_means = []

for group_name in metadata_continuous:
    # Remove the metadata not relevant to this grouping
    groups = df[['sample-id', 'variable', 'value', group_name]]

    # Calculate the means accross iterations
    agger = {'value': 'mean', group_name: 'first'}
    groups = groups.groupby(['sample-id', 'variable']).agg(agger).reset_index()

    # Add a column to store the errors
    groups = groups.assign(Error=groups.value)

    # Group by metadata value and calculate the means and error
    agger = {'Error': 'sem', 'value': 'mean'}
    group = groups.groupby([group_name, 'variable']).agg(agger).reset_index()

    # Assign information for grouping
    group_names = [group_name for x in group[group_name]]
    group = group.assign(GroupName=group_names)

    # Rename columns and append to the list of dataframes
    new_names = {
        'variable': 'SamplingDepth',
        'value': 'AverageValue',
         group_name: 'Grouping'
    }
    group_means.append(group.rename(index=str, columns=new_names))

# Stack all the different groups into a single dataframe
df = pd.concat(group_means, axis=0, sort=False)
df.SamplingDepth = df.SamplingDepth.astype(float)
df.Error = df.Error.astype(float)
df.AverageValue = df.AverageValue.astype(float)
df.GroupName = df.GroupName.astype(str)

In [ ]:
%%R -i df

pd <- position_dodge(width = 50)
df.var <- subset(df, GroupName=="DaysSinceExperimentStart")
p <- ggplot(data = df.var, aes(x = SamplingDepth, y = AverageValue, color = Grouping,group=Grouping)) +
      geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
      geom_point(stat='identity', position = pd, size = 1) +
      geom_line(stat='identity', position = pd) +
      labs(title = 'Alpha Diversity',
           subtitle = 'Grouped by DaysSinceExperimentStart') +
      theme_bw() +
      theme(legend.position = 'bottom',
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5)) + scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

# Save plots
ggsave('faith_pd_DaysSinceExperimentStart.png', height = 6, width = 6)

In [ ]:
Image("faith_pd_DaysSinceExperimentStart.png")

\pagebreak

In [ ]:
%%R -i df

pd <- position_dodge(width = 50)
df.var <- subset(df, GroupName=="Depth")
p <- ggplot(data = df.var, aes(x = SamplingDepth, y = AverageValue, color = Grouping,group=Grouping)) +
      geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
      geom_point(stat='identity', position = pd, size = 1) +
      geom_line(stat='identity', position = pd) +
      labs(title = 'Alpha Diversity',
           subtitle = 'Grouped by Depth') +
      theme_bw() +
      theme(legend.position = 'bottom',
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5)) + scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

# Save plots
ggsave('faith_pd_Depth.png', height = 6, width = 6)

In [ ]:
Image("faith_pd_Depth.png")

\pagebreak

In [ ]:
# Read in the data
df = pd.read_csv('faith_pd.csv', sep=',')

# Reshape the data into (mostly) long format
headers = list(set(mdf.columns).intersection(set(df.columns)))
df = df.melt(id_vars=['sample-id'] + headers)

# Remove info on specific iterations to allow for grouping by value
replacements = {x:int(x.split('_')[0].split('-')[1]) for x in df.variable.unique()}
df.variable.replace(replacements, inplace=True)

# For storing
group_means = []

for group_name in metadata_discrete:
    # Remove the metadata not relevant to this grouping
    groups = df[['sample-id', 'variable', 'value', group_name]]

    # Calculate the means accross iterations
    agger = {'value': 'mean', group_name: 'first'}
    groups = groups.groupby(['sample-id', 'variable']).agg(agger).reset_index()

    # Add a column to store the errors
    groups = groups.assign(Error=groups.value)

    # Group by metadata value and calculate the means and error
    agger = {'Error': 'sem', 'value': 'mean'}
    group = groups.groupby([group_name, 'variable']).agg(agger).reset_index()

    # Assign information for the colors
    colors = [color_maps[group_name][str(x)] for x in group[group_name]]
    group = group.assign(GroupID=colors)

    # Assign information for grouping
    group_names = [group_name for x in group[group_name]]
    group = group.assign(GroupName=group_names)

    # Rename columns and append to the list of dataframes
    new_names = {
        'variable': 'SamplingDepth',
        'value': 'AverageValue',
         group_name: 'Grouping'
    }
    group_means.append(group.rename(index=str, columns=new_names))

# Stack all the different groups into a single dataframe
df = pd.concat(group_means, axis=0, sort=False)
df.SamplingDepth = df.SamplingDepth.astype(float)
df.Error = df.Error.astype(float)
df.AverageValue = df.AverageValue.astype(float)
df.Grouping = df.Grouping.astype(str)
df.GroupID = df.GroupID.astype(str)
df.GroupName = df.GroupName.astype(str)

In [ ]:
%%R -i df
pd <- position_dodge(width = 50)

p <- ggplot(data = df, aes(x = SamplingDepth, y = AverageValue, color = GroupID)) +
     geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
     geom_point(stat='identity', position = pd, size = 1) +
     geom_line(stat='identity', position = pd) +
     facet_wrap(~GroupName) + colFill + colScale +
     labs(title = 'Alpha Diversity',
          subtitle = 'Grouped by Discrete Metadata Categories') +
     theme_bw() +
     theme(legend.position = 'none',
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5))

# Save plots
ggsave('faith_pd_Depth.png', height = 6, width = 6)

In [ ]:
Image("faith_pd_Depth.png")

The above plot represents the average value of alpha diversity at each sampling depth. The error bars show the standard error within each group. Groups are determined by the metadata value in each category specified in the plot.

\pagebreak

## Observed ASV

In [ ]:
# Read in the data
df = pd.read_csv('observed_features.csv', sep=',')

# Reshape the data into (mostly) long format
headers = list(set(mdf.columns).intersection(set(df.columns)))
df = df.melt(id_vars=['sample-id'] + headers)

# Remove info on specific iterations to allow for grouping by value
replacements = {x:int(x.split('_')[0].split('-')[1]) for x in df.variable.unique()}
df.variable.replace(replacements, inplace=True)

# For storing
group_means = []

for group_name in metadata_continuous:
    # Remove the metadata not relevant to this grouping
    groups = df[['sample-id', 'variable', 'value', group_name]]

    # Calculate the means accross iterations
    agger = {'value': 'mean', group_name: 'first'}
    groups = groups.groupby(['sample-id', 'variable']).agg(agger).reset_index()

    # Add a column to store the errors
    groups = groups.assign(Error=groups.value)

    # Group by metadata value and calculate the means and error
    agger = {'Error': 'sem', 'value': 'mean'}
    group = groups.groupby([group_name, 'variable']).agg(agger).reset_index()

    # Assign information for grouping
    group_names = [group_name for x in group[group_name]]
    group = group.assign(GroupName=group_names)

    # Rename columns and append to the list of dataframes
    new_names = {
        'variable': 'SamplingDepth',
        'value': 'AverageValue',
         group_name: 'Grouping'
    }
    group_means.append(group.rename(index=str, columns=new_names))

# Stack all the different groups into a single dataframe
df = pd.concat(group_means, axis=0, sort=False)
df.SamplingDepth = df.SamplingDepth.astype(float)
df.Error = df.Error.astype(float)
df.AverageValue = df.AverageValue.astype(float)
df.GroupName = df.GroupName.astype(str)

In [ ]:
%%R -i df

pd <- position_dodge(width = 50)
df.var <- subset(df, GroupName=="DaysSinceExperimentStart")
p <- ggplot(data = df.var, aes(x = SamplingDepth, y = AverageValue, color = Grouping,group=Grouping)) +
      geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
      geom_point(stat='identity', position = pd, size = 1) +
      geom_line(stat='identity', position = pd) +
      labs(title = 'Alpha Diversity',
           subtitle = 'Grouped by DaysSinceExperimentStart') +
      theme_bw() +
      theme(legend.position = 'bottom',
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5)) + scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

# Save plots
ggsave('observed_features_DaysSinceExperimentStart.png', height = 6, width = 6)

In [ ]:
Image("observed_features_DaysSinceExperimentStart.png")

\pagebreak

In [ ]:
%%R -i df

pd <- position_dodge(width = 50)
df.var <- subset(df, GroupName=="Depth")
p <- ggplot(data = df.var, aes(x = SamplingDepth, y = AverageValue, color = Grouping,group=Grouping)) +
      geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
      geom_point(stat='identity', position = pd, size = 1) +
      geom_line(stat='identity', position = pd) +
      labs(title = 'Alpha Diversity',
           subtitle = 'Grouped by Depth') +
      theme_bw() +
      theme(legend.position = 'bottom',
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5)) + scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

# Save plots
ggsave('observed_features_Depth.png', height = 6, width = 6)

In [ ]:
Image("observed_features_Depth.png")

\pagebreak

In [ ]:
# Read in the data
df = pd.read_csv('observed_features.csv', sep=',')

# Reshape the data into (mostly) long format
headers = list(set(mdf.columns).intersection(set(df.columns)))
df = df.melt(id_vars=['sample-id'] + headers)

# Remove info on specific iterations to allow for grouping by value
replacements = {x:int(x.split('_')[0].split('-')[1]) for x in df.variable.unique()}
df.variable.replace(replacements, inplace=True)

# For storing
group_means = []

for group_name in metadata_discrete:
    # Remove the metadata not relevant to this grouping
    groups = df[['sample-id', 'variable', 'value', group_name]]

    # Calculate the means accross iterations
    agger = {'value': 'mean', group_name: 'first'}
    groups = groups.groupby(['sample-id', 'variable']).agg(agger).reset_index()

    # Add a column to store the errors
    groups = groups.assign(Error=groups.value)

    # Group by metadata value and calculate the means and error
    agger = {'Error': 'sem', 'value': 'mean'}
    group = groups.groupby([group_name, 'variable']).agg(agger).reset_index()

    # Assign information for the colors
    colors = [color_maps[group_name][str(x)] for x in group[group_name]]
    group = group.assign(GroupID=colors)

    # Assign information for grouping
    group_names = [group_name for x in group[group_name]]
    group = group.assign(GroupName=group_names)

    # Rename columns and append to the list of dataframes
    new_names = {
        'variable': 'SamplingDepth',
        'value': 'AverageValue',
         group_name: 'Grouping'
    }
    group_means.append(group.rename(index=str, columns=new_names))

# Stack all the different groups into a single dataframe
df = pd.concat(group_means, axis=0, sort=False)
df.SamplingDepth = df.SamplingDepth.astype(float)
df.Error = df.Error.astype(float)
df.AverageValue = df.AverageValue.astype(float)
df.Grouping = df.Grouping.astype(str)
df.GroupID = df.GroupID.astype(str)
df.GroupName = df.GroupName.astype(str)

In [ ]:
%%R -i df
pd <- position_dodge(width = 50)

p <- ggplot(data = df, aes(x = SamplingDepth, y = AverageValue, color = GroupID)) +
     geom_errorbar(aes(ymin=AverageValue-Error, ymax=AverageValue+Error), width=100, position = pd) +
     geom_point(stat='identity', position = pd, size = 1) +
     geom_line(stat='identity', position = pd) +
     facet_wrap(~GroupName) + colFill + colScale +
     labs(title = 'Alpha Diversity',
          subtitle = 'Grouped by Discrete Metadata Categories') +
     theme_bw() +
     theme(legend.position = 'none',
           plot.title = element_text(hjust = 0.5),
           plot.subtitle = element_text(hjust = 0.5))

# Save plots
ggsave('observed_features_Depth.png', height = 6, width = 6)

In [ ]:
Image("observed_features_Depth.png")

The above plot represents the average value of alpha diversity at each sampling depth. The error bars show the standard error within each group. Groups are determined by the metadata value in each category specified in the plot.

\pagebreak

In [ ]:
# Reset this variable
latex = []
with open('mod_revtex.tplx', 'r') as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    if '((* endblock packages *))' in line:
        packages_end = i

# Define the colors for each category/value in the alpha/beta plots
for group_name, group in df.groupby('GroupName'):
    zipped = zip(group.GroupID.unique(), group.Grouping.unique())
    lists = ['{}/{}'.format(x, y) for x, y in zipped]
    latex.append('\\def\\{group}{{{colors}}}\n'.format(group=group_name, colors=','.join(lists)))

new_lines = lines[:packages_end] + latex + lines[packages_end:]

with open('mod_revtex.tplx', 'w') as f:
    for line in new_lines:
        f.write(line)

# Beta Diversity

## Interpreting Beta Diversity Results

Beta diversity estimates how similar or dissimilar samples are based on their microbiome composition. Different to alpha diversity, which is estimated per sample, beta diversity is a distance that is calculated between pairs of samples. Samples that are similar to each other in their microbiome composition will have a low distance between them based on beta diversity, while those that are very different in their composition will have a large distance. Principal Coordinate Analysis (PCoA) is an ordination technique that visually represents the samples based on their beta diversity distances to facilitate the identification of clusters or gradients of samples. By default, the first three principal coordinates are shown in PCoA plots.

## Bray-Curtis, grouped by DaysSinceExperimentStart

In [ ]:
import pandas as pd
with open('bray_curtis_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign variable to DataFrame
df = df.assign(variable=mdf['DaysSinceExperimentStart'][[x for x in df.axes[0]]])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('bray_curtis_pcoa_results-DaysSinceExperimentStart.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             legend = 4,
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$variable, label = rownames(df)), alpha = 1.0) +
         theme_bw() +
         theme(legend.position = 'bottom',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by DaysSinceExperimentStart') +
         scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('bray_curtis_pcoa_results-DaysSinceExperimentStart-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'bottom',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by DaysSinceExperimentStart') +
                      scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("bray_curtis_pcoa_results-DaysSinceExperimentStart.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("bray_curtis_pcoa_results-DaysSinceExperimentStart-PC1-PC2.png")

In [ ]:
Image("bray_curtis_pcoa_results-DaysSinceExperimentStart-PC1-PC3.png")

In [ ]:
Image("bray_curtis_pcoa_results-DaysSinceExperimentStart-PC2-PC3.png")

\pagebreak

## Bray-Curtis, grouped by Depth

In [ ]:
import pandas as pd
with open('bray_curtis_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign variable to DataFrame
df = df.assign(variable=mdf['Depth'][[x for x in df.axes[0]]])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('bray_curtis_pcoa_results-Depth.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             legend = 4,
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$variable, label = rownames(df)), alpha = 1.0) +
         theme_bw() +
         theme(legend.position = 'bottom',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by Depth') +
         scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('bray_curtis_pcoa_results-Depth-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'bottom',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by Depth') +
                      scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("bray_curtis_pcoa_results-Depth.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("bray_curtis_pcoa_results-Depth-PC1-PC2.png")

In [ ]:
Image("bray_curtis_pcoa_results-Depth-PC1-PC3.png")

In [ ]:
Image("bray_curtis_pcoa_results-Depth-PC2-PC3.png")

\pagebreak

## Bray-Curtis, grouped by Ethnicity

In [ ]:
import pandas as pd
with open('bray_curtis_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['Ethnicity'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['Ethnicity'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('bray_curtis_pcoa_results-Ethnicity.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by Ethnicity')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('bray_curtis_pcoa_results-Ethnicity-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by Ethnicity')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("bray_curtis_pcoa_results-Ethnicity.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("bray_curtis_pcoa_results-Ethnicity-PC1-PC2.png")

In [ ]:
Image("bray_curtis_pcoa_results-Ethnicity-PC1-PC3.png")

In [ ]:
Image("bray_curtis_pcoa_results-Ethnicity-PC2-PC3.png")

\pagebreak

## Bray-Curtis, grouped by InterventionCode

In [ ]:
import pandas as pd
with open('bray_curtis_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['InterventionCode'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['InterventionCode'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('bray_curtis_pcoa_results-InterventionCode.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by InterventionCode')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('bray_curtis_pcoa_results-InterventionCode-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by InterventionCode')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("bray_curtis_pcoa_results-InterventionCode.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("bray_curtis_pcoa_results-InterventionCode-PC1-PC2.png")

In [ ]:
Image("bray_curtis_pcoa_results-InterventionCode-PC1-PC3.png")

In [ ]:
Image("bray_curtis_pcoa_results-InterventionCode-PC2-PC3.png")

\pagebreak

## Bray-Curtis, grouped by SpecimenBodySite

In [ ]:
import pandas as pd
with open('bray_curtis_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['SpecimenBodySite'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['SpecimenBodySite'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('bray_curtis_pcoa_results-SpecimenBodySite.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by SpecimenBodySite')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('bray_curtis_pcoa_results-SpecimenBodySite-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by SpecimenBodySite')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("bray_curtis_pcoa_results-SpecimenBodySite.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("bray_curtis_pcoa_results-SpecimenBodySite-PC1-PC2.png")

In [ ]:
Image("bray_curtis_pcoa_results-SpecimenBodySite-PC1-PC3.png")

In [ ]:
Image("bray_curtis_pcoa_results-SpecimenBodySite-PC2-PC3.png")

\pagebreak

## Bray-Curtis, grouped by SpecimenType

In [ ]:
import pandas as pd
with open('bray_curtis_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['SpecimenType'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['SpecimenType'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('bray_curtis_pcoa_results-SpecimenType.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by SpecimenType')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('bray_curtis_pcoa_results-SpecimenType-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by SpecimenType')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("bray_curtis_pcoa_results-SpecimenType.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("bray_curtis_pcoa_results-SpecimenType-PC1-PC2.png")

In [ ]:
Image("bray_curtis_pcoa_results-SpecimenType-PC1-PC3.png")

In [ ]:
Image("bray_curtis_pcoa_results-SpecimenType-PC2-PC3.png")

\pagebreak

## Unweighted UniFrac, grouped by DaysSinceExperimentStart

In [ ]:
import pandas as pd
with open('unweighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign variable to DataFrame
df = df.assign(variable=mdf['DaysSinceExperimentStart'][[x for x in df.axes[0]]])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('unweighted_unifrac_pcoa_results-DaysSinceExperimentStart.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             legend = 4,
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$variable, label = rownames(df)), alpha = 1.0) +
         theme_bw() +
         theme(legend.position = 'bottom',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by DaysSinceExperimentStart') +
         scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('unweighted_unifrac_pcoa_results-DaysSinceExperimentStart-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'bottom',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by DaysSinceExperimentStart') +
                      scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("unweighted_unifrac_pcoa_results-DaysSinceExperimentStart.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("unweighted_unifrac_pcoa_results-DaysSinceExperimentStart-PC1-PC2.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-DaysSinceExperimentStart-PC1-PC3.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-DaysSinceExperimentStart-PC2-PC3.png")

\pagebreak

## Unweighted UniFrac, grouped by Depth

In [ ]:
import pandas as pd
with open('unweighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign variable to DataFrame
df = df.assign(variable=mdf['Depth'][[x for x in df.axes[0]]])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('unweighted_unifrac_pcoa_results-Depth.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             legend = 4,
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$variable, label = rownames(df)), alpha = 1.0) +
         theme_bw() +
         theme(legend.position = 'bottom',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by Depth') +
         scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('unweighted_unifrac_pcoa_results-Depth-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'bottom',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by Depth') +
                      scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("unweighted_unifrac_pcoa_results-Depth.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("unweighted_unifrac_pcoa_results-Depth-PC1-PC2.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-Depth-PC1-PC3.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-Depth-PC2-PC3.png")

\pagebreak

## Unweighted UniFrac, grouped by Ethnicity

In [ ]:
import pandas as pd
with open('unweighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['Ethnicity'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['Ethnicity'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('unweighted_unifrac_pcoa_results-Ethnicity.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by Ethnicity')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('unweighted_unifrac_pcoa_results-Ethnicity-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by Ethnicity')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("unweighted_unifrac_pcoa_results-Ethnicity.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("unweighted_unifrac_pcoa_results-Ethnicity-PC1-PC2.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-Ethnicity-PC1-PC3.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-Ethnicity-PC2-PC3.png")

\pagebreak

## Unweighted UniFrac, grouped by InterventionCode

In [ ]:
import pandas as pd
with open('unweighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['InterventionCode'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['InterventionCode'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('unweighted_unifrac_pcoa_results-InterventionCode.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by InterventionCode')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('unweighted_unifrac_pcoa_results-InterventionCode-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by InterventionCode')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("unweighted_unifrac_pcoa_results-InterventionCode.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("unweighted_unifrac_pcoa_results-InterventionCode-PC1-PC2.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-InterventionCode-PC1-PC3.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-InterventionCode-PC2-PC3.png")

\pagebreak

## Unweighted UniFrac, grouped by SpecimenBodySite

In [ ]:
import pandas as pd
with open('unweighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['SpecimenBodySite'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['SpecimenBodySite'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('unweighted_unifrac_pcoa_results-SpecimenBodySite.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by SpecimenBodySite')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('unweighted_unifrac_pcoa_results-SpecimenBodySite-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by SpecimenBodySite')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("unweighted_unifrac_pcoa_results-SpecimenBodySite.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("unweighted_unifrac_pcoa_results-SpecimenBodySite-PC1-PC2.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-SpecimenBodySite-PC1-PC3.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-SpecimenBodySite-PC2-PC3.png")

\pagebreak

## Unweighted UniFrac, grouped by SpecimenType

In [ ]:
import pandas as pd
with open('unweighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['SpecimenType'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['SpecimenType'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('unweighted_unifrac_pcoa_results-SpecimenType.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by SpecimenType')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('unweighted_unifrac_pcoa_results-SpecimenType-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by SpecimenType')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("unweighted_unifrac_pcoa_results-SpecimenType.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("unweighted_unifrac_pcoa_results-SpecimenType-PC1-PC2.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-SpecimenType-PC1-PC3.png")

In [ ]:
Image("unweighted_unifrac_pcoa_results-SpecimenType-PC2-PC3.png")

\pagebreak

## Weighted UniFrac, grouped by DaysSinceExperimentStart

In [ ]:
import pandas as pd
with open('weighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign variable to DataFrame
df = df.assign(variable=mdf['DaysSinceExperimentStart'][[x for x in df.axes[0]]])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('weighted_unifrac_pcoa_results-DaysSinceExperimentStart.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             legend = 4,
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$variable, label = rownames(df)), alpha = 1.0) +
         theme_bw() +
         theme(legend.position = 'bottom',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by DaysSinceExperimentStart') +
         scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('weighted_unifrac_pcoa_results-DaysSinceExperimentStart-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'bottom',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by DaysSinceExperimentStart') +
                      scale_color_gradient(low="#0000FF", high="#FF0000", name = 'DaysSinceExperimentStart', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("weighted_unifrac_pcoa_results-DaysSinceExperimentStart.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("weighted_unifrac_pcoa_results-DaysSinceExperimentStart-PC1-PC2.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-DaysSinceExperimentStart-PC1-PC3.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-DaysSinceExperimentStart-PC2-PC3.png")

\pagebreak

## Weighted UniFrac, grouped by Depth

In [ ]:
import pandas as pd
with open('weighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign variable to DataFrame
df = df.assign(variable=mdf['Depth'][[x for x in df.axes[0]]])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('weighted_unifrac_pcoa_results-Depth.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             legend = 4,
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$variable, label = rownames(df)), alpha = 1.0) +
         theme_bw() +
         theme(legend.position = 'bottom',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by Depth') +
         scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")

print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('weighted_unifrac_pcoa_results-Depth-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'bottom',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by Depth') +
                      scale_color_gradient(low="#FF9900", high="#9900CC", name = 'Depth', space = "Lab", na.value = "#888888", guide = "colorbar", aesthetics = "color")
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("weighted_unifrac_pcoa_results-Depth.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("weighted_unifrac_pcoa_results-Depth-PC1-PC2.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-Depth-PC1-PC3.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-Depth-PC2-PC3.png")

\pagebreak

## Weighted UniFrac, grouped by Ethnicity

In [ ]:
import pandas as pd
with open('weighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['Ethnicity'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['Ethnicity'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('weighted_unifrac_pcoa_results-Ethnicity.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by Ethnicity')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('weighted_unifrac_pcoa_results-Ethnicity-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by Ethnicity')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("weighted_unifrac_pcoa_results-Ethnicity.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("weighted_unifrac_pcoa_results-Ethnicity-PC1-PC2.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-Ethnicity-PC1-PC3.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-Ethnicity-PC2-PC3.png")

\pagebreak

## Weighted UniFrac, grouped by InterventionCode

In [ ]:
import pandas as pd
with open('weighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['InterventionCode'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['InterventionCode'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('weighted_unifrac_pcoa_results-InterventionCode.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by InterventionCode')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('weighted_unifrac_pcoa_results-InterventionCode-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by InterventionCode')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("weighted_unifrac_pcoa_results-InterventionCode.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("weighted_unifrac_pcoa_results-InterventionCode-PC1-PC2.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-InterventionCode-PC1-PC3.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-InterventionCode-PC2-PC3.png")

\pagebreak

## Weighted UniFrac, grouped by SpecimenBodySite

In [ ]:
import pandas as pd
with open('weighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['SpecimenBodySite'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['SpecimenBodySite'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('weighted_unifrac_pcoa_results-SpecimenBodySite.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by SpecimenBodySite')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('weighted_unifrac_pcoa_results-SpecimenBodySite-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by SpecimenBodySite')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("weighted_unifrac_pcoa_results-SpecimenBodySite.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("weighted_unifrac_pcoa_results-SpecimenBodySite-PC1-PC2.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-SpecimenBodySite-PC1-PC3.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-SpecimenBodySite-PC2-PC3.png")

\pagebreak

## Weighted UniFrac, grouped by SpecimenType

In [ ]:
import pandas as pd
with open('weighted_unifrac_pcoa_results.txt') as f:
    page = f.read()

store = {}
# Parse the PCA information file
for i, line in enumerate(page.split('\n')):
    parts = line.split('\t')
    if i == 0:
        length = int(parts[1])
    if i > 9 :
        if line == '':
            break
        store[parts[0]] = list(map(float, parts[1:length]))

# Create a dataframe and name the axes
df = pd.DataFrame.from_dict(store).T
cols = {x:'PC{}'.format(x + 1) for x in df.columns}
df = df.rename(index=str, columns=cols)

# Assign GroupIDs based on metadata
samples = mdf['SpecimenType'][[x for x in df.axes[0]]]
df = df.assign(GroupID=[color_maps['SpecimenType'][str(x)] for x in samples])

In [ ]:
%%R -i df

# Create the plots for the first three PCs
png('weighted_unifrac_pcoa_results-SpecimenType.png', width = 6, height = 6, unit='in', res=200)
p <- ggpairs(df[,c(1:3)],
             upper = list(continuous = "points", combo = "box_no_facet"),
             lower = list(continuous = "points", combo = "dot_no_facet"),
             aes(color = df$GroupID, label = rownames(df), alpha=0.5)) +
         theme_bw() +
         theme(legend.position = 'none',
               plot.title = element_text(hjust = 0.5),
               plot.subtitle = element_text(hjust = 0.5)) +
         labs(title = 'PCA plot',
              subtitle = 'Colored by SpecimenType')

# Add the color palette to each of the plots
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        p[i,j] <- p[i,j] + colScale + colFill
    }
}
print(p)
out <- dev.off()

# Print the individual PCA plots with labels
for(i in 1:p$nrow) {
    for(j in 1:p$ncol){
        # Only print the PCAs not the frequency distributions
        if (i > 2 && j < 3 || i > 1 && j < 2) {
            # Setup and save each individual PCA plot
            filename <- sprintf('weighted_unifrac_pcoa_results-SpecimenType-%s-%s.png',
                                p[i, j]$labels$x,
                                p[i, j]$labels$y)
            png(filename, width = 6, height = 6, unit='in', res=200)
            sp <- p[i,j] + geom_text_repel(color = "black", alpha = 0.35, max.overlaps = Inf) +
                      theme(legend.position = 'none',
                            plot.title = element_text(hjust = 0.5),
                            plot.subtitle = element_text(hjust = 0.5)) +
                      labs(title = sprintf('%s vs. %s',
                                           p[i, j]$labels$x,
                                           p[i, j]$labels$y),
                           subtitle = 'Colored by SpecimenType')
            print(sp)
            out <- dev.off()
        }
    }
}

In [ ]:
Image("weighted_unifrac_pcoa_results-SpecimenType.png")

The above plot represents the first three compenents created when performing Principle Component Analysis on the Beta diversity of the samples.

In [ ]:
Image("weighted_unifrac_pcoa_results-SpecimenType-PC1-PC2.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-SpecimenType-PC1-PC3.png")

In [ ]:
Image("weighted_unifrac_pcoa_results-SpecimenType-PC2-PC3.png")

\pagebreak